In [1]:
import langsmith
from langchain import chat_models, prompts, smith
from langchain.schema import output_parser
from langchain.chat_models import AzureChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
llm = AzureChatOpenAI(
    azure_deployment="chat",
    openai_api_type="azure",
    temperature=0.0
)

c:\Users\nttLu\.virtualenvs\genai-articles-wuG-FtGG\Lib\site-packages\langchain\chat_models\azure_openai.py:162: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
c:\Users\nttLu\.virtualenvs\genai-articles-wuG-FtGG\Lib\site-packages\langchain\chat_models\azure_openai.py:170: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.com/openai/deployments/example-deployment. Updating https://cog-um7r24s27bps6.openai.azure.com/openai to https://cog-um7r24s27bps6.openai.azure.com/openai.
  warnings.warn(


In [3]:
from langchain.schema import HumanMessage

message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
llm([message])

AIMessage(content="J'aime programmer.")

In [27]:
dataset_name = "ds-elderly-sea-70"

In [4]:
# Filter runs to add to the dataset
client = langsmith.Client()
project_name = "my-chain-example"

runs = client.list_runs(
    project_name= project_name,
    execution_order=1,
    error=False,
)

dataset = client.create_dataset(dataset_name, description="An example dataset")
for run in runs:
    client.create_example(
        inputs=run.inputs,
        outputs=run.outputs,
        dataset_id=dataset.id,
    )

In [41]:

eval_config = smith.RunEvalConfig(
    evaluators=[
        "cot_qa",
        smith.RunEvalConfig.LabeledCriteria("relevance"),
        smith.RunEvalConfig.LabeledCriteria("coherence"),
        smith.RunEvalConfig.LabeledCriteria("helpfulness"),
        smith.RunEvalConfig.LabeledCriteria("maliciousness"),
        smith.RunEvalConfig.LabeledCriteria("misogyny"),
        smith.RunEvalConfig.Criteria(
            {
                "Useful": "Read the question and check if the answer is useful. At the end can I use the answer to solve my problem?"
                "Respond Y if it is, N if it's not."
            }
        ),
    ],
    custom_evaluators=[],
    eval_llm=llm
)


In [42]:
import uuid
guid = str(uuid.uuid4())


client = langsmith.Client()
chain_results = client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=llm,
    evaluation=eval_config,
    project_name=f"test-giving-lord-{guid}",
    concurrency_level=5,
    verbose=True,
)
    

 

View the evaluation results for project 'test-giving-lord-950311aa-dc64-4759-8a16-1edd9270a0ee' at:
https://smith.langchain.com/o/1a3b231d-e8b9-4837-8416-2403db894308/projects/p/f847c72e-c1e3-49b5-9da3-f65b1032094a?eval=true

View all tests for Dataset ds-elderly-sea-70 at:
https://smith.langchain.com/o/1a3b231d-e8b9-4837-8416-2403db894308/datasets/6346d51b-f5f3-439e-82c1-a84d9f663ca8
[------------------------------------------------->] 1/1